# Use Function Calling / Tool Use to orchestrate sub-agents and knowledge bases 



Function calling is another powerful approach to orchestrate multi-agent collaboration within a hierarchical model. This method involves providing the primary Large Language Model (LLM) with a set of predefined functions that represent the capabilities of various sub-agents and knowledge bases. The LLM can then decide which function to call based on the user's input and the task at hand. This approach allows for more structured and controlled interactions between the main orchestrator and its sub-components, ensuring that each specialized agent or knowledge base is utilized effectively.

In this notebook, you will 
* Use the sub-agents built in [01_create_agents_and_kbs.ipynb](./01_create_agents_and_kbs.ipynb)
* Define a tool class and tool specifications
* Utilize the Amazon Bedrock Converse API to orchestrate routing and requests



First step is to install the pre-requisites packages. NOTE: You only need to do this is this is the first notebook you are running. 

In [1]:
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore awscli 

In [ ]:
import boto3
import logging
from typing import List, Dict
import json
import uuid

%load_ext autoreload
%autoreload 2

from knowledge_base import BedrockKnowledgeBase
from agent import AgentsForAmazonBedrock

In [ ]:
boto3.__version__

In [53]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [54]:
agents = AgentsForAmazonBedrock()

In [55]:
region = agents.get_region()
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"

Identify the list of FMs you'd like to experiment with for function-calling based
approach to orchestrating available sub-agents and knowledge bases. 
Only include models that your account has access to use, and that support Bedrock Converse 
tool use. The underlying sub-agents will continue to use whichever models they were created with.

In [192]:
foundation_models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "mistral.mistral-large-2402-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0"
    #"anthropic.claude-3-5-sonnet-20240620-v1:0"
    ]

In [57]:
bedrock_client = boto3.client('bedrock-runtime')

In [ ]:
from knowledge_base import BedrockKnowledgeBaseHelper
helper = BedrockKnowledgeBaseHelper()
kb_id = helper.get_kb()
kb_arn = f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"

print(kb_id)
print(kb_arn)

Define a class that acts as a pass-through for executing the Tool Use calls that 
the Bedrock Converse API returns based on the user request. It includes one method for 
each sub-agent, and one method for knowledge base lookups.

In [201]:
class MortgageTools:
    def __init__(self, session_id: str="9999999", session_state: dict={}, 
                 mortgage_kb_id: str=kb_id, kb_model_id: str=foundation_models[0]):
        self._session_id = session_id
        self._session_state = session_state
        self._existing_mortgage_agent_id = agents.get_agent_id_by_name('existing_mortgage_agent')
        self._mortgage_application_agent_id = agents.get_agent_id_by_name('mortgage_application_agent')
        self._kb_model_arn = f"arn:aws:bedrock:{region}::foundation-model/{kb_model_id}"
        self._mortgage_kb_id = mortgage_kb_id

    def invoke_existing_mortgage_agent(self, input_text):
        return agents.invoke(input_text, self._existing_mortgage_agent_id,
                            session_id=self._session_id, session_state=self._session_state)
    def invoke_mortgage_application_agent(self, input_text):
        return agents.invoke(input_text, self._mortgage_application_agent_id,
                            session_id=self._session_id, session_state=self._session_state)
    def perform_kb_lookup(self, input_text):
        kb_response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={"text": input_text},
            retrieveAndGenerateConfiguration={
                    "type": "KNOWLEDGE_BASE",
                    "knowledgeBaseConfiguration": {
                        'knowledgeBaseId': self._mortgage_kb_id,
                        "modelArn": self._kb_model_arn,
                        "retrievalConfiguration": {
                            "vectorSearchConfiguration": {
                                "numberOfResults":5
                            } 
                        }
                    }
                }
            )
        return kb_response['output']['text']
    def set_session(self, session_id, session_state: dict={}):
        self._session_id = session_id
        self._session_state = session_state

Now test the MortgageTools class directly to ensure it can handle the requests we will
later get from Converse.

In [202]:
tools = MortgageTools("999")

In [ ]:
getattr(tools, "invoke_existing_mortgage_agent")("I'm customer 99. When's my pmt due?")

In [ ]:
getattr(tools, "invoke_mortgage_application_agent")("I am customer 99. What docs do I owe you for my existing application?")

In [ ]:
getattr(tools, "perform_kb_lookup")("compare and contrast 15-year vs 30-year mortgage type")

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state 

In [207]:
tools.set_session("893", session_state)

In [ ]:
getattr(tools, "invoke_existing_mortgage_agent")("what's my balance, and how many years until I'm done w/ this sill mortgage?")

In [ ]:
supervisor_agent_name = "mortgage_supervisor_agent"
function_defs = agents.get_function_defs(supervisor_agent_name)
function_defs 

In [210]:
def map_function_defs_to_tools(function_defs: Dict) -> Dict:
    tools = []
    for function_def in function_defs:
        params_as_properties = {}
        required_params = []
        for param_name, param_details in function_def["parameters"].items():
            params_as_properties[param_name] = {
                "type": param_details["type"],
                "description": param_details["description"],
            }
            if param_details["required"]:
                required_params.append(param_name)
        tool_def = {
            "toolSpec": {
                "name": function_def["name"].replace("-", "_"),
                "description": function_def["description"],
                'inputSchema': {
                    'json': {
                        'type': 'object',
                            'properties': params_as_properties,
                            'required': required_params
                    }
                }
            }
        }
        tools.append(tool_def)
    return tools

In [ ]:
map_function_defs_to_tools(function_defs['functions'])

In [212]:
sub_agent_tools = map_function_defs_to_tools(function_defs['functions'])

Now create a tool spec for the knowledge base, and assemble the full set of tools.

In [213]:
kb_tool = {'toolSpec': {'name': 'perform_kb_lookup',
   'description': 'handle general mortgage questions about different mortgage types like comparing 15-year vs 30-year',
   'inputSchema': {'json': {'type': 'object',
     'properties': {'input_text': {'type': 'string',
       'description': 'The request to be answered by doing a knowledge base lookup.'}},
     'required': ['input_text']}}}}

In [214]:
full_tool_set = []
full_tool_set.extend(sub_agent_tools)
full_tool_set.append(kb_tool)

## Converse API

In this lab we will be utilizing the Converse API. You can use the Amazon Bedrock Converse API to create conversational applications that send and receive messages to and from an Amazon Bedrock model. For example, you can create a chat bot that maintains a conversation over many turns and uses a persona or tone customization that is unique to your needs, such as a helpful technical support assistant. The Converse API allows you to utilizes tools with your models. For example, you might have a chat application that lets users find out out the most popular song played on a radio station. To answer a request for the most popular song, a model needs a tool that can query and return the song information. 

In [ ]:
toolConfigGen = {
    'tools': full_tool_set,
    'toolChoice': {
        'auto': {},
        }
}
toolConfigGen

In [216]:
#Function for caling the Bedrock Converse API...
def converse_with_tools(messages, system='', model_id=foundation_models[0], 
                        tool_config=toolConfigGen, bedrock_client=bedrock_client):
    response = bedrock_client.converse(
        modelId=model_id,
        system=system,
        messages=messages,
        toolConfig=tool_config
    )
    return response

In [217]:
#Function for orchestrating the conversation flow...
def converse(prompt, system=' ', bedrock_client=bedrock_client, 
             session_id: str=None, tool_config=toolConfigGen,
             verbose=False, respond_direct_from_tool=False,
             model_id:str=foundation_models[0],
             tools: MortgageTools=tools):
    #Add the initial prompt:
    messages = []
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    )

    if verbose:
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Initial prompt:\n{json.dumps(messages, indent=2)}")

    #Invoke the model the first time:
    output = converse_with_tools(messages, system, tool_config=tool_config, model_id=model_id)
    if verbose:
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Output so far:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}")

    #Add the intermediate output to the prompt:
    messages.append(output['output']['message'])

    function_calling = next((c['toolUse'] for c in output['output']['message']['content'] if 'toolUse' in c), None)

    #Check if function calling is triggered:
    if not function_calling:
        return output['output']['message']['content'][0]['text']
    else:
        #Get the tool name and arguments:
        tool_name = function_calling['name']
        tool_args = function_calling['input'] or {}
        
        #Run the tool:
        if session_id is not None:
            tools.set_session(session_id)
        if verbose:
            print(f"\n{datetime.now().strftime('%H:%M:%S')} - Running ({tool_name}) tool...")
            
        tool_response = getattr(tools, tool_name)(**tool_args)

        if verbose: 
            print(tool_response)

        if respond_direct_from_tool: 
            return tool_response
        else:
            #Add the tool result to the prompt:
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            'toolResult': {
                                'toolUseId':function_calling['toolUseId'],
                                'content': [
                                    {
                                        "text": tool_response
                                    }
                                ]
                            }
                        }
                    ]
                }
            )
            if verbose:
                print(f"\n{datetime.now().strftime('%H:%M:%S')} - Messages so far:\n{json.dumps(messages, indent=2)}")

            # Invoke the model one more time:
            output = converse_with_tools(messages, system, bedrock_client=bedrock_client, model_id=model_id)
            if verbose:
                print(f"\n{datetime.now().strftime('%H:%M:%S')} - Final output:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}\n")
            if len(output['output']['message']['content']) > 0:
                return output['output']['message']['content'][0]['text']
            else:
                return "None"

In [ ]:
user_request = "I'm customer 99. what's my balance?"

session_id:str = str(uuid.uuid1())
output = converse(
        system = [{"text": "You are a virtual agent that delegates requests to sub-agents. "}],
        prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      "Always try to use an available tool instead of asking a clarifying question."
                      f"User request: {user_request}",
        session_id=session_id,
        respond_direct_from_tool=True,
        verbose=False,
        model_id=foundation_models[0])
print(f"Output: {output}") 

In [ ]:
user_request = "What are the key differences between 15-year and 30-year mortgages?"

session_id:str = str(uuid.uuid1())
output = converse(
        system = [{"text": "You are a virtual agent that delegates requests to sub-agents. "}],
        prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      "Always try to use an available tool instead of asking a clarifying question."
                      f"User request: {user_request}",
        session_id=session_id,
        respond_direct_from_tool=True,
        verbose=False,
        model_id=foundation_models[0])
print(f"Output: {output}") 

In [ ]:
user_request = "I'm customer 984. What documents do I still owe for my new mortgage application?"

session_id:str = str(uuid.uuid1())
output = converse(
        system = [{"text": "You are a virtual agent that delegates requests to sub-agents. "}],
        prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      "Always try to use an available tool instead of asking a clarifying question."
                      f"User request: {user_request}",
        session_id=session_id,
        respond_direct_from_tool=True,
        verbose=False,
        model_id=foundation_models[0])
print(f"Output: {output}") 

In [ ]:
user_request = "What is your favorite basketball team?"

session_id:str = str(uuid.uuid1())
output = converse(
        system = [{"text": "You are a virtual agent that delegates requests to sub-agents. "}],
        prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      "Always try to use an available tool instead of asking a clarifying question."
                      f"User request: {user_request}",
        session_id=session_id,
        respond_direct_from_tool=True,
        verbose=False,
        model_id=foundation_models[0])
print(f"Output: {output}") 

In [ ]:
user_request = "I'm customer 99. what's my balance?"

for m in foundation_models:
    print(f"\n**** Using function calling via model: {m} ****")
    for i in range(3):
        session_id:str = str(uuid.uuid1())
        output = converse(
            system = [{"text": "You are a virtual agent that delegates requests to sub-agents."}],
            prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      "Always try to use an available tool instead of asking a clarifying question."
                      f"User request: {user_request}",
            session_id=session_id,
            respond_direct_from_tool=True,
            verbose=False,
            model_id=m)
        print(f"{i+1}) Output: {output}") 

## Quick Performance Test

Performance is a consideration when deciding on using a supervisor agent, intent classification, etc. In the below code you will see the impact of function calling where functions will be invoked multiple times to obtain averages for latency for each response to provide you an idea of it's overall impact to performance. Latency will vary based on model types you select for each agent and the integrations you utilize within your AWS Lambda functions.

In [223]:
import uuid 
import time
import numpy as np

def query_loop_by_function_calling(user_request, num_invokes, fc_model):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()

        _output = converse(
            system = [{"text": "You are a virtual agent that delegates requests to sub-agents."}],
            prompt =  "Instructions: Respond as though you are completing the requests on your own," +\
                      "without mentioning antying about your sub-agents." +\
                      f"User request: {user_request}",
            session_id=_session_id,
            respond_direct_from_tool=True,
            verbose=False,
            model_id=fc_model)

        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked by function calling {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [ ]:
query_loop_by_function_calling("I am customer 999. how many years until the mortgage maturity date?", 
                               25,
                               fc_model=foundation_models[0])